In [1]:
#read csv and load into dataframe

import pandas as pd
file_path = '/Users/taief/Desktop/Music_Final/labeled_lyrics_cleaned.csv'

lyrics_df = pd.read_csv('labeled_lyrics_cleaned.csv')
lyrics_df.head(20)

,Unnamed: 0,artist,seq,song,label
0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday,0.6260
1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.6300
2,2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside,0.2400
3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot,0.5360
4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.3710
5,5,Elijah Blake,I just want to ready your mind\r\n'Cause I'll ...,Uno,0.3210
6,6,Elijah Harris,To believe\r\nOr not to believe\r\nThat is the...,Girlfriend (Main),0.6010
7,7,Elis,Dieses ist lange her.\r\nDa ich deine schmalen...,Abendlied,0.3330
8,8,Elis,A child is born\r\nOut of the womb of a mother...,Child,0.5060
9,9,Elis,Out of the darkness you came \r\nYou looked so...,Come to Me,0.1790


In [7]:
import pandas as pd

# Assuming you've already read the CSV into a DataFrame named 'lyrics_df'
# If not, you can uncomment the next line to read it from a file
# lyrics_df = pd.read_csv('/path/to/your/data.csv')

# Check if 'Unnamed: 0' exists in the DataFrame and drop it if present
if 'Unnamed: 0' in lyrics_df.columns:
    lyrics_df = lyrics_df.drop('Unnamed: 0', axis=1)

# Create a copy to avoid SettingWithCopyWarning when dropping na
lyrics_df_cleaned = lyrics_df.copy()
lyrics_df_cleaned.dropna(inplace=True)

# Normalize text in 'seq'
lyrics_df_cleaned['seq'] = lyrics_df_cleaned['seq'].str.lower()\
                                                    .replace(r'[\r\n]', ' ', regex=True)\
                                                    .replace('[^a-z ]', '', regex=True)

# Remove duplicates based on 'artist' and 'song'
lyrics_df_cleaned = lyrics_df_cleaned.drop_duplicates(subset=['artist', 'song'])

# Reset the index of the DataFrame after dropping duplicates
lyrics_df_cleaned.reset_index(drop=True, inplace=True)

# Quick look at the cleaned data
print(lyrics_df_cleaned.head())

#save the clean df
lyrics_df_cleaned.to_csv('/Users/taief/Desktop/Music_Final/data_cleaned.csv', index=False)


         artist                                                seq  \
0  Elijah Blake  no no  i aint ever trapped out the bando  but ...   
1  Elijah Blake  the drinks go down and smoke goes up i feel my...   
2  Elijah Blake  she dont live on planet earth no more  she fou...   
3  Elijah Blake  trippin off that grigio mobbin lights low  tri...   
4  Elijah Blake  i see a midnight panther so gallant and so bra...   

                 song  label  
0            Everyday  0.626  
1    Live Till We Die  0.630  
2       The Otherside  0.240  
3               Pinot  0.536  
4  Shadows & Diamonds  0.371  


In [11]:
import pandas as pd

# Load the dataset
file_path = '/Users/taief/Desktop/Music_Final/data.csv'
data_final = pd.read_csv(file_path)

# Extract and clean artist names from data_final
# Since artists are stored in a list-like format as strings, we'll extract them properly
import ast  # Import Abstract Syntax Trees to safely evaluate strings containing Python literals
data_final['artists'] = data_final['artists'].apply(ast.literal_eval).apply(lambda x: [artist.lower() for artist in x])

# Unpack the list of artists and get unique artists in lower case
unique_artists = set(artist for sublist in data_final['artists'] for artist in sublist)

In [13]:
# Convert artist names in lyrics_df_cleaned to lower case for matching
lyrics_df_cleaned['artist'] = lyrics_df_cleaned['artist'].str.lower()

# Filter lyrics_df_cleaned to keep rows where the artist is in unique_artists list
lyrics_df_filtered = lyrics_df_cleaned[lyrics_df_cleaned['artist'].isin(unique_artists)]

# Display the first few rows of the filtered DataFrame
print(lyrics_df_filtered.head(20))

# Count the number of unique artists in the filtered lyrics DataFrame
unique_artists_count = lyrics_df_filtered['artist'].nunique()
print(f"Number of unique artists in the filtered DataFrame: {unique_artists_count}")

# Save the filtered DataFrame to a CSV file
lyrics_df_filtered.to_csv('/Users/taief/Desktop/Music_Final/filtered_lyrics.csv', index=False)


         artist                                                seq  \
30  elis regina  deve ser bom deve ser bom  to fechado pra bala...   
31  elis regina  once there was a way to get back homeward  onc...   
32  elis regina  gracias a la vida que me ha dado tanto  me dio...   
33  elis regina  sad is to live in solitude  far from your tran...   
34  elis regina  let someone start believing in you  let him ho...   
35        elisa  tell me who you are now what youre waiting for...   
36        elisa  tic tac tic tac  run up those stairs cause som...   
37        elisa  why do you wanna know everything why do you ne...   
38        elisa  what shall i do  just to feel a little over ze...   
39        elisa  cover of velvet underground  lou reed  here sh...   
40        elisa  i heard there was a secret chord that david pl...   
41        elisa  tell me about the place youre from  tell me ab...   
42        elisa  a shower of pure joy  a shower of pure joy  a ...   
43        elisa  tog

In [14]:
#track name and artist name 